In [3]:
# import stanfordnlp
# !pip3 install stanfordnlp
# stanfordnlp.download('hi')

In [101]:
# morph_file = open("hindi_data/hin.tok.txt").read().split("\n")
# for line in morph_file:
#     word = line.strip("\n")
#     doc = nlp(word)
# #     print(doc.sentences[0].words[0].text, "\t", doc.sentences[0].words[0].upos)
#     print(*[f'word: {word.text+" "}\tupos: {word.upos}\txpos: {word.xpos}' for sent in doc.sentences for word in sent.words], sep='\n')
#     print("\n")

In [12]:
doc = nlp(tok_sents[0])
print(*[word.text+"\t"+word.upos+"\n" for sent in doc.sentences for word in sent.words])
tags = [word.upos for sent in doc.sentences for word in sent.words]
tags

लेकिन	CCONJ
 गांव	NOUN
 के	ADP
 जगदीश	PROPN
 मेघवाल	PROPN
 ,	PUNCT
 मोहन	PROPN
 .	PUNCT
 .	PUNCT
 .	PUNCT



['CCONJ',
 'NOUN',
 'ADP',
 'PROPN',
 'PROPN',
 'PUNCT',
 'PROPN',
 'PUNCT',
 'PUNCT',
 'PUNCT']

In [97]:
import stanza
from collections import defaultdict

2021-07-30 10:09:29 INFO: Loading these models for language: hi (Hindi):
| Processor | Package |
-----------------------
| tokenize  | hdtb    |
| pos       | hdtb    |

2021-07-30 10:09:29 INFO: Use device: cpu
2021-07-30 10:09:29 INFO: Loading: tokenize
2021-07-30 10:09:29 INFO: Loading: pos
2021-07-30 10:09:30 INFO: Done loading processors!


In [ ]:
nlp = stanza.Pipeline('hi', processors='tokenize,pos')

In [98]:
suff_tag = {"NOUN":"NSUFF", "VERB":"VSUFF", "ADJ":"AJSUFF"}
prefs = {"अ", "अनु", "वि", "पर", "सद", "बद", "दुर", "उप"}

In [30]:
tok_sents = open("hindi_data/hin.tok.txt").read().split("\n")
morph_file = open("hindi_data/hin.segmented.EOLmarked.txt").read().split("\nE OL\n")
print(morph_file[:5])

In [99]:
def update_tag_dist(morphs, tags, tag_dist):
    for tidx, tag in enumerate(tags):
        morph = morphs[tidx].split(" ")
        if len(morph)==1:
            tag_dist[morph[0]][tag] += 1
        if len(morph)>1:
#             if len(morph[0])<4:
#                 print(morph[0])
            if morph[0] in prefs:
                tag_dist[morph[0]]["PREF"] += 1
                morph = morph[1:]
            
            for m in morph[:-1]:
                if len(m)!=1:
                    tag_dist[m][tag] += 1
            if tag in suff_tag:
                tag_dist[morph[-1]][suff_tag[tag]]+=1
            else:
                tag_dist[morph[-1]][tag] += 1
                
    return tag_dist
    

In [100]:
tag_dist = defaultdict(lambda: defaultdict(lambda: 0))
count = 0
for sidx, sent in enumerate(tok_sents[:150000]):
    doc = nlp(sent)
    tags = [word.upos for sent in doc.sentences for word in sent.words]
    morphs = morph_file[sidx].split("\n")
    if(len(tags)!=len(morphs)):
        count += 1
        continue
#         print(sent)
#         print(morphs)
#         print(len(sent.split()))
#         print(len(morphs))
#         print(len(tags))
#         print([word.text for sent in doc.sentences for word in sent.words])
#         break
    tag_dist = update_tag_dist(morphs, tags, tag_dist)
    
    

In [51]:
count

In [54]:
sidx

18863

In [102]:
len(tag_dist)

35263

In [56]:
list(tag_dist.items())[200:300]

[('सकता',
  defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
              {'AUX': 380, 'VERB': 4})),
 ('ओह',
  defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
              {'PROPN': 3, 'NOUN': 1, 'SCONJ': 1})),
 ('ियो',
  defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
              {'PROPN': 6,
               'NSUFF': 8,
               'AJSUFF': 1,
               'AUX': 1,
               'ADJ': 1,
               'NOUN': 1})),
 ('समर्थन',
  defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>, {'NOUN': 59})),
 ('कैलकुल',
  defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>, {'NOUN': 1})),
 ('ेटर',
  defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>, {'NSUFF': 2})),
 ('68',
  defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
              {'NUM': 4, 'PROPN': 1})),
 ('कां',
  defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
              {'NOUN': 6, 'PRON': 1, 'ADJ': 1, 'PROPN': 7, 'SCONJ': 1

In [57]:
token_dist = defaultdict(lambda: defaultdict(lambda: 0))
for token in tag_dist:
    for tag in tag_dist[token]:
        token_dist[tag][token] = tag_dist[token][tag]

In [89]:
sorted(list(token_dist["NOUN"].items()), key = lambda x: x[1], reverse = True)[:100]

[('बात', 568),
 ('देश', 495),
 ('लोगों', 428),
 ('काम', 397),
 ('आज', 394),
 ('दिन', 392),
 ('समय', 378),
 ('लोग', 375),
 ('तरह', 359),
 ('बार', 342),
 ('घर', 333),
 ('नाम', 310),
 ('सरकार', 304),
 ('साल', 296),
 ('रूप', 259),
 ('वर्ष', 253),
 ('कार्य', 228),
 ('जीवन', 227),
 ('क्षेत्र', 225),
 ('फिल्म', 220),
 ('पुलिस', 213),
 ('शुरू', 189),
 ('पता', 187),
 ('पानी', 185),
 ('दुनिया', 174),
 ('समाज', 169),
 ('राज्य', 168),
 ('बस', 165),
 ('प्रकार', 158),
 ('मन', 154),
 ('चुनाव', 142),
 ('जगह', 142),
 ('परिवार', 142),
 ('विकास', 138),
 ('&', 137),
 ('हाथ', 135),
 ('दिनों', 135),
 ('अधिकार', 134),
 ('गांव', 130),
 ('जनता', 129),
 ('सेवा', 127),
 ('रात', 127),
 ('पार्टी', 126),
 ('व्यक्ति', 125),
 ('शहर', 125),
 ('टीम', 121),
 ('स्थिति', 121),
 ('शब्द', 120),
 ('महिला', 120),
 ('ध्यान', 117),
 ('याद', 116),
 ('शिक्षा', 115),
 ('कविता', 115),
 ('अध्यक्ष', 115),
 ('विचार', 115),
 ('धर्म', 110),
 ('स्थान', 110),
 ('मामले', 109),
 ('कारण', 108),
 ('रंग', 108),
 ('जानकारी', 108),
 ('सवाल', 108

In [103]:
len(token_dist["NSUFF"])

2519

In [96]:
tag_dist["रंग"]

defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
            {'NOUN': 108, 'PROPN': 12, 'VERB': 1, 'AUX': 1})

In [104]:
import json
with open("tag_distributions/tag_dist_h.json", "w") as l_file:
    json.dump(tag_dist, l_file, ensure_ascii=False)

In [105]:
len(tag_dist)

35263